In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
file_name = "1a700c7d-59f7-48f3-913a-a09cfc08e3bb.csv"
df = pd.read_csv(file_name)

print("--- Tarihsel Veri Setinin İlk 5 Satırı ---")
print(df.head())
print("\n")

--- Tarihsel Veri Setinin İlk 5 Satırı ---
   _id   Yil  Benzinli - Istanbul  Benzinli - Turkiye  Dizel - Istanbul  \
0    1  2020               149712              502665            181402   
1    2  2021               214775              627164            136437   
2    3  2022               236324              767943            123383   
3    4  2023               380861             1431184            162418   
4    5  2024               383786             1605139            131466   

   Dizel - Turkiye  Hibrit - Istanbul  Hibrit - Turkiye  Elektrik - Istanbul  \
0           464291              12777             20049                 2468   
1           409581              31522             53312                 5959   
2           379516              27770             48118                21527   
3           509053              43476             87939                46389   
4           456947              72056            169439                59799   

   Elektrik - Turkiye  Lp

In [3]:
forecast_cols = df.columns.drop(['_id', 'Yil']).tolist()
forecast_df = pd.DataFrame()
n_periods = 5  # Tahmin edilecek yıl sayısı (2025'ten 2029'a kadar)

In [4]:
X_train = df['Yil'].values.reshape(-1, 1)

In [5]:
future_years = np.array(range(df['Yil'].max() + 1, df['Yil'].max() + 1 + n_periods)).reshape(-1, 1)

In [6]:
print("--- 5 Yıllık Tahmin İşlemi Başlatılıyor (Doğrusal Regresyon) ---")

--- 5 Yıllık Tahmin İşlemi Başlatılıyor (Doğrusal Regresyon) ---


In [7]:
for col in forecast_cols:
    y_train = df[col].values

    # Modeli Başlatma ve Eğitme
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Gelecek Yıllar İçin Tahmin Yapma
    forecast_values = model.predict(future_years)
    
    # Negatif değerleri 0 olarak düzeltme (Özellikle LPG gibi düşüş trendindeki yakıtlar için)
    # Araç sayısı 0'ın altında olamaz.
    forecast_values[forecast_values < 0] = 0
    
    # Tahmin sonuçlarını en yakın tam sayıya yuvarlama
    forecast_values = forecast_values.round().astype(int)

    # Sonuçları Kaydetme
    if forecast_df.empty:
        forecast_df['Yil'] = future_years.flatten()
        forecast_df[col] = forecast_values
    else:
        forecast_df[col] = forecast_values

In [8]:
if col == 'Benzinli - Turkiye':
        plt.figure(figsize=(12, 7))
        
        # Tarihsel Veri
        plt.scatter(df['Yil'], df[col], color='blue', label='Tarihsel Veri')
        
        # Eğilim Çizgisi (Regresyon)
        # Tüm yıllar için eğilim çizgisini çiziyoruz
        X_all = np.concatenate((X_train, future_years))
        y_all_pred = model.predict(X_all)
        y_all_pred[y_all_pred < 0] = 0 # Görselleştirmede de 0 sınırı
        plt.plot(X_all, y_all_pred, color='orange', linestyle='-', label='Doğrusal Eğilim Çizgisi')
        
        # Tahmin Edilen Veri Noktaları
        plt.scatter(forecast_df['Yil'], forecast_df[col], color='red', label='Tahmin Edilen Veri (2025-2029)')
        
        plt.title(f"{col} Kayıt Sayısı Tahmini (Doğrusal Regresyon)", fontsize=16)
        plt.xlabel('Yıl', fontsize=14)
        plt.ylabel('Kaydedilen Araç Sayısı', fontsize=14)
        plt.legend()
        plt.grid(True)
        plt.xticks(X_all.flatten())
        plt.tight_layout()
        plt.show() # Jupyter uygulamasında figür gösterilir
        plt.savefig("Benzinli_Turkiye_Forecast_Linear.png") # Dosyayı kaydet

In [9]:
df_historical = df.drop(columns=['_id'])
df_historical = df_historical.set_index('Yil')
forecast_df_final = forecast_df.set_index('Yil')

combined_df = pd.concat([df_historical, forecast_df_final])

forecast_output_file = "Motorlu_Arac_Tahminleri_2025_2029.csv"
forecast_df_final.to_csv(forecast_output_file)

print("\n--- Tarihsel ve Tahmin Edilen Verilerin Tamamı (2020-2029) ---")
print(combined_df)

print(f"\n✅ Tahmin verileri {forecast_output_file} dosyasına kaydedilmiştir.")


--- Tarihsel ve Tahmin Edilen Verilerin Tamamı (2020-2029) ---
      Benzinli - Istanbul  Benzinli - Turkiye  Dizel - Istanbul  \
Yil                                                               
2020               149712              502665            181402   
2021               214775              627164            136437   
2022               236324              767943            123383   
2023               380861             1431184            162418   
2024               383786             1605139            131466   
2025               463362             1889509            124854   
2026               526785             2190406            117465   
2027               590209             2491303            110076   
2028               653632             2792200            102687   
2029               717055             3093097             95298   

      Dizel - Turkiye  Hibrit - Istanbul  Hibrit - Turkiye  \
Yil                                                          
2020   